# Implement Low-Rank Adaptation for LLMs in PyTorch

https://lightning.ai/lightning-ai/studios/code-lora-from-scratch?tab=overview&layout=column&path=cloudspaces%2F01hm9hypqc6y1hrapb5prmtz0h&y=3&x=0

# 1 - Understanding LoRA

Pretrained LLMs are often dubbed foundation models because of their versatility across various tasks. However, it is often useful to adapt a pretrained LLM for a specific dataset or task, which we can accomplish via finetuning.

Finetuning allows the model to adapt to specific domains without costly pretraining, yet updating all layers can still be computationally expensvie, espcially for larger models.

LoRA offers a more efficient alternative to regular finetuning. As discussed in more detail in the paper [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685), LoRA approximates a layer's weight changes during training, $\Delta W$, in a low-rank format.

for instance, whereas in regular finetuning, we compute the weight updates of a weight matrix $W$ as $\Delta W$, in LoRA, we approximate $\Delta W$ through the matrix multiplication of two smaller matrices $A B$, as illustrated in the figure below. (If you are familiar with PCA or SVD, consider this as decomposing $\Delta W$ in $A$ and $B$)

<table>
    <tr>
        <td><img src="./images_3/lora_summary.png" width="800"/></td>
    </tr>
</table>

Note that $r$, in the figure above, is a hyperparameter here that we can use to specify the rank of the low-rank matrices used for adaptation. A smaller $r$ leads to a simpler low-rank matrix, which results in fewer parameters to learn during adaptation. This can lead to faster training and potentially reduced computational requirements. However, with a smaller $r$, the capacity of the low-rank matrix to capture task specific information decreases.

To provide a concrete example, suppose the weight matrix $W$ of a given layer has a size $5,000 \times 10,000$ (50M parameters in total). If we choose a rank $r=8$, we initialize two matrices: the $5,000 \times 8$-dimensional matrix $B$ and the $8 \times 10,000$-dimensional matrix $A$. Added together, $A$ and $B$ have only $40,000 + 80,000 = 120,000$ parameters, which is 400 times smaller than the 50M parameters in regular finetuning via $\Delta W$.

In practice, it's important to experiment with different $r$ values to find the right balance to achieve the desired performance on the new task.

# 2 - Coding LoRA from scratch

Since conceptual explanations can sometimes be abstract, let's now implement LoRA ourselves to get a btter idea of how it works. In code, we can implement a LoRA layer as follows:

In [1]:
import torch


class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.W_a = torch.nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.W_b = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.W_a @ self.W_b)
        return x

In the code above, `in_dim` is the input dimension of the layer we want to modify using LoRA, and `out_dim` is the respective output dimension of that layer.

We previously discussed that the rank of the matrices $A$ and $B$ is a hyperparamter that controls the complexity and the number of additional parameters introduced by LoRA.

However, looking at the code above, we added another hyperparameter, the scaling factor `alpha`. This factor determines the magnitude of the changes introduced by the LoRA layer to the model's existing weights: `alpha * (x @ A @ B)`. A higher value of `alpha` means larger adjustments to the model's behaviour, while a lower value results in more subtle changes.

Another thing to note is that we initialized `A` with small values from a random distribution. Here, the standard deviation of this distribution is determined by the square root of the rank (this choice ensures that the initial values in `A` are not too large). However, we initialized `B` with zeros. The rationale here is that at the beginning of the training, before `A` and `B` are updated via backpropagation, the `LoRALayer` does not impact the original weights because `AB=0` if `B=0`

Note that LoRA is usually applied to a neural network's linear (feedforward) layers. For example, suppose we have a simple PyTorch model or module with two linear layers (e.g., this could be the feedforward module of a transformer block). And suppose this module's forward method looks like as follows:

```python

def forward(self, x):
    x = self.linear_1(x)
    x = F.relu(x)
    x = self.linear_2(x)
    return x

```

If we use LoRA, we'd add the LoRA updates to these linear layer outputs as follows:

```python

def forward(self, x):
    x = self.linear_1(x) + self.lora_1(x)
    x = F.relu(x)
    x = self.linear_2(x) + self.lora_2(x)
    return logits

```

In code, when implementing LoRA by modifying existing PyTorch models, an easy way to implement such a LoRA-modification of the linear layers is to replace each Linear layer with a `LinearWithLoRA` layer that combines the `Linear` layer with our previous `LoRALayer` implementation:

In [2]:
class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

## 2.1 - Example

In [3]:
torch.manual_seed(123)

# a simple linear layer with 10 inputs and 1 output
# requires_grad=False makes it non-trainable
linear_layer = torch.nn.Linear(10, 1)
linear_layer.weight.requires_grad = False
linear_layer.bias.requires_grad = False

# a simple example input
x = torch.rand((1, 10))

linear_layer(x)

tensor([[-0.5745]])

Replace linear layer with LoRA layer

In [4]:
lora_layer = LinearWithLoRA(linear=linear_layer, rank=8, alpha=1)
lora_layer(x)

tensor([[-0.5745]], grad_fn=<AddBackward0>)

Note that the LoRA layer will not change the linear output untils it is trained because its `W_b` weight matrix is initialized to all zeros. For LoRA to take effect, we have to train the model. Below is a simple toy example.

Let's simulate a simple weight update:

In [5]:
lora_layer.lora.W_b = torch.nn.Parameter(lora_layer.lora.W_b + 0.01 * x[0])


We can now see that the output has changed:

In [6]:
lora_layer(x)

tensor([[-0.5863, -0.5758, -0.5779, -0.5800, -0.5814, -0.5766, -0.5887, -0.5811,
         -0.5859, -0.5892]], grad_fn=<AddBackward0>)

## 2.2 - Summary

These concepts above are summarized in the following figure:

<table>
    <tr>
        <td><img src="./images_3/lora_from_scratch_summary.jpg" width="1000"/></td>
    </tr>
</table>

In practice, to equip and finetune a model with LoRA, all we have to do is replace its pretrained `Linear` layers with our new `LinearWithLoRA` layer. 

# 3 - Finetuning with LoRA -- A hands-on example

LoRA is a method that can be applied to various types of neural networks, not just generative models like GPT or image generation models. For this hands-on example, we will train a small BERT model for text classification because classification accuracy is easier to evaluate than generated text.

In particular, we will use a pretrained DistilBERT model from the transformers library:

```python

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)
    
```

Since we only want to train the new LoRA weights, we freeze all model parameters by setting `requires_grad` to `False` for all trainable parameters:

```python

for param in model.parameters():
    param.requires_grad = False

```

## 3.1 - Loading the dataset into DataFrames

In [7]:
import os
from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import pandas as pd
import torch

/anaconda/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


For the data, we are going to use a sampled version of the IMDB dataset that we have prepared by running the `notes-ai/Foundation models/Fine-tuning/Parameter-efficient Finetuning/local_dataset_utilities.py` script.

Since this notebook is also a modified version of another Lightning blog post that uses the same data, I have prepared a sampled version of the dataset to test it

In [8]:
df_train = pd.read_csv("train_sample.csv")
df_val = pd.read_csv("val_sample.csv")
df_test = pd.read_csv("test_sample.csv")

In [9]:
imdb_dataset = load_dataset(
    "csv",
    data_files={
        "train": "train_sample.csv",
        "validation": "val_sample.csv",
        "test": "test_sample.csv",
    },
)

print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'label'],
        num_rows: 3500
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'text', 'label'],
        num_rows: 500
    })
})


## 3.2 - Tokenization and numericalization

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [11]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [12]:
imdb_tokenized = imdb_dataset.map(tokenize_text, batched=True, batch_size=None)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 3947.25 examples/s]

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 3720.76 examples/s]


In [13]:
del imdb_dataset

In [14]:
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [15]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## 3.3 - Set up DataLoaders

In [16]:
from torch.utils.data import DataLoader, Dataset


class IMDBDataset(Dataset):
    def __init__(self, dataset_dict, partition_key="train"):
        self.partition = dataset_dict[partition_key]

    def __getitem__(self, index):
        return self.partition[index]

    def __len__(self):
        return self.partition.num_rows

In [17]:
train_dataset = IMDBDataset(imdb_tokenized, partition_key="train")
val_dataset = IMDBDataset(imdb_tokenized, partition_key="validation")
test_dataset = IMDBDataset(imdb_tokenized, partition_key="test")

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=12,
    shuffle=True, 
    num_workers=4
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=12,
    num_workers=4
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=12,
    num_workers=4
)

## 3.4 - Prepare model

### 3.4.1 - Initialize DistilBERT

In [18]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 3.4.2 - Freeze all layers

In [19]:
for param in model.parameters():
    param.requires_grad = False

### 3.4.3 - Add LoRA layers

Based on the output below, we can see that the model consists of 6 transformer layers containing `Linear` layers and two `Linear` output layers:

In [20]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

We can selectively enable LoRA for these `Linear` layers by defining the following assignment function and loop:

In [21]:
from functools import partial

lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_query = True
lora_key = False
lora_value = True
lora_projection = False
lora_mlp = False
lora_head = False

layers = []

assign_lora = partial(LinearWithLoRA, rank=lora_r, alpha=lora_alpha)

for layer in model.distilbert.transformer.layer:
    if lora_query:
        layer.attention.q_lin = assign_lora(layer.attention.q_lin)
    if lora_key:
        layer.attention.k_lin = assign_lora(layer.attention.k_lin)
    if lora_value:
        layer.attention.v_lin = assign_lora(layer.attention.v_lin)
    if lora_projection:
        layer.attention.out_lin = assign_lora(layer.attention.out_lin)
    if lora_mlp:
        layer.ffn.lin1 = assign_lora(layer.ffn.lin1)
        layer.ffn.lin2 = assign_lora(layer.ffn.lin2)
if lora_head:
    model.pre_classifier = assign_lora(model.pre_classifier)
    model.classifier = assign_lora(model.classifier)

As we can see below, the `Linear` layers have successfully been replaced by the `LinearWithLoRA` layers.

In [22]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): LinearWithLoRA(
              (linear): Linear(in_features=768, out_features=768, bias=True)
              (lora): LoRALayer()
            )
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): LinearWithLoRA(
              (linear): Linear(in_features=768, out_features=768, bias=True)
              (lora): LoRALayer()
            )
            (out_lin): Linear(in_features=768, out_features=768, bias

In [23]:
# Check if linear layers are frozen
for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")

distilbert.embeddings.word_embeddings.weight: False
distilbert.embeddings.position_embeddings.weight: False
distilbert.embeddings.LayerNorm.weight: False
distilbert.embeddings.LayerNorm.bias: False
distilbert.transformer.layer.0.attention.q_lin.linear.weight: False
distilbert.transformer.layer.0.attention.q_lin.linear.bias: False
distilbert.transformer.layer.0.attention.q_lin.lora.W_a: True
distilbert.transformer.layer.0.attention.q_lin.lora.W_b: True
distilbert.transformer.layer.0.attention.k_lin.weight: False
distilbert.transformer.layer.0.attention.k_lin.bias: False
distilbert.transformer.layer.0.attention.v_lin.linear.weight: False
distilbert.transformer.layer.0.attention.v_lin.linear.bias: False
distilbert.transformer.layer.0.attention.v_lin.lora.W_a: True
distilbert.transformer.layer.0.attention.v_lin.lora.W_b: True
distilbert.transformer.layer.0.attention.out_lin.weight: False
distilbert.transformer.layer.0.attention.out_lin.bias: False
distilbert.transformer.layer.0.sa_layer_no

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print("Total number of trainable parameters:", count_parameters(model))

Total number of trainable parameters: 147456


## 3.5 - Finetuning

In [25]:
from local_model_utilities import CustomLightningModule

lightning_model = CustomLightningModule(model)

In [26]:
callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )  # save top 1 model
]
logger = CSVLogger(save_dir="logs/", name="my-model")

In [27]:
trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="gpu",
    precision="16-mixed",
    devices=1,
    logger=logger,
    log_every_n_steps=10,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [28]:
import time
start = time.time()

trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

Missing logger folder: logs/my-model
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                                | Params
-----------------------------------------------------------------
0 | model    | DistilBertForSequenceClassification | 67.1 M
1 | val_acc  | MulticlassAccuracy                  | 0     
2 | test_acc | MulticlassAccuracy                  | 0     
-----------------------------------------------------------------
147 K     Trainable params
67.0 M    Non-trainable params
67.1 M    Total params
268.410   Total estimated model params size (MB)


Epoch 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 292/292 [00:46<00:00,  6.21it/s, v_num=0, val_loss=0.313, val_acc=0.872]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 292/292 [00:46<00:00,  6.21it/s, v_num=0, val_loss=0.313, val_acc=0.872]


In [29]:
train_acc = trainer.test(lightning_model, dataloaders=train_loader, ckpt_path="best", verbose=False)
val_acc = trainer.test(lightning_model, dataloaders=val_loader, ckpt_path="best", verbose=False)
test_acc = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best", verbose=False)

Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=1-step=584.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=1-step=584.ckpt
/anaconda/envs/pytorch/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 292/292 [00:18<00:00, 16.13it/s]


Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=1-step=584.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=1-step=584.ckpt


Testing DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:02<00:00, 15.59it/s]


Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=1-step=584.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=1-step=584.ckpt


Testing DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:02<00:00, 15.24it/s]


In [30]:
print(f"Train acc: {train_acc[0]['accuracy']*100:2.2f}%")
print(f"Val acc:   {val_acc[0]['accuracy']*100:2.2f}%")
print(f"Test acc:  {test_acc[0]['accuracy']*100:2.2f}%")

Train acc: 94.80%
Val acc:   89.60%
Test acc:  83.80%


In [31]:
import shutil

# Cleanup checkpoint files as we don't need them later
log_dir = f"logs/my-model"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

# 4 - Optimizing the LoRA configuration

In [32]:
lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_query = True
lora_key = False
lora_value = True
lora_projection = False
lora_mlp = False
lora_head = False

Note that this only involves applying LoRA to an attention layer's query and value weight matrices. Optionally, we can enable LoRA for other layers as well. Furthermore, we can control the number of trainable parameters in each LoRA layer by modifying the rank (`lora_r`).

One way to improve the LoRA performance is to manually fiddle with these hyperparameter choices. However, to make the hyperparameter tuning more convenient you can use use a "grid search" approach.

You can test it, by running the `gridsearch.py` script, which runs the following hyperparameter grid on all available GPUs:

```python

alpha_values = [1, 4, 8, 16, 32, 64]
rank_values = [1, 2, 4, 8, 16, 32]
devices = range(torch.cuda.device_count())
lora_query = ["True"]
lora_key = ["False", "True"]
lora_value = ["True"]
lora_projection = ["False", "True"]
lora_mlp = ["False", "True"]
lora_head = ["False", "True"]

```